# G123 window size calibration

In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

# General setup parameters.
atlas_id = "agam"
analysis_version = "dev"
cohorts_analysis = "20240924"
contigs = ["3L"]
dask_scheduler = "single-threaded"

# Other parameters.
sample_sets = "AG1000G-BF-A"
sample_query = None
min_cohort_size = 20
max_cohort_size = 50
cohort_id = "BF-09_Houet_colu_2012_Q3"
h12_calibration_contig = "3L"

## Setup

In [ ]:
import pandas as pd
import numpy as np
import yaml
from selection_atlas.setup import AtlasSetup

# Initialise the atlas setup.
setup = AtlasSetup(
    atlas_id=atlas_id,
    analysis_version=analysis_version,
    cohorts_analysis=cohorts_analysis,
    contigs=contigs,
    dask_scheduler=dask_scheduler,
)

In [ ]:
sample_sets

In [ ]:
df_cohorts = pd.read_csv(setup.cohorts_file).set_index("cohort_id")
df_cohorts.head()

In [ ]:
cohort = df_cohorts.loc[cohort_id]
cohort

In [ ]:
# determine the phasing analysis to use
cohort.taxon

In [ ]:
sample_query = cohort.sample_query
sample_query

In [ ]:
if cohort.taxon == "arabiensis":
    sites = site_mask = "arab"
else:
    sites = site_mask = "gamb_colu"
site_mask

In [ ]:
h12_calibration_contig

In [ ]:
window_sizes = (200, 500, 1000, 2000, 5000, 10000)
window_sizes

## Run calibration

In [ ]:
setup.malariagen_api.plot_g123_calibration(
    contig=h12_calibration_contig,
    site_mask=site_mask,
    sites=sites,
    sample_sets=sample_sets,
    sample_query=sample_query,
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);

In [ ]:
calibration_runs = setup.malariagen_api.g123_calibration(
    contig=h12_calibration_contig,
    site_mask=site_mask,
    sites=sites,
    sample_sets=sample_sets,
    sample_query=sample_query,
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
)
calibration_runs

In [ ]:
selected_window_size = None
for window_size in window_sizes:
    x = calibration_runs[str(window_size)]
    x95 = np.percentile(x, 95)
    if x95 < 0.1:
        selected_window_size = window_size
        break
selected_window_size

## Write outputs

In [ ]:
output = {"g123_window_size": selected_window_size}
g123_calibration_file = setup.g123_calibration_files.as_posix().format(cohort=cohort_id)
with open(g123_calibration_file, mode="w") as output_file:
    yaml.safe_dump(output, output_file)